# Assigment 1, $\color{red}{\text{Step 3}}$ 
**Framework:**
* Please discard the network (cupper‐plate system).
* Please discard input data related to the upward/downward reserve cost and the maximum upward/downward reserve provision capability. These input data will be used later in Step 4.
* Please assume there is no intra‐day market. There are day‐ahead and then balancing markets.
* Demands in the day‐ahead market are inelastic to price.
    
**Procedure:**
1. Clear the day‐ahead market.
2. In the balancing stage, generator 7 becomes out of the system (unexpected outage). The actual production of farms 1 and 2 is 10% lower than their day‐ahead forecast, while that of farms 3 and 4 is 25% higher than their day‐ahead forecast. All remaining 11 conventional generators (but not demands) are potential balancing service providers. There is no other balancing service provider (except for involuntarily load curtailment). Each conventional generator as the balancing service provider offers the upward balancing service at a price equal to the day‐ahead price plus 10% of her production cost. Similarly, she offers the downward balancing service (if she can depending on her day‐ahead schedule) at a price equal to the day‐ahead price minus 12% of her production cost. Clear the balancing market, and derive the balancing price.
3. Calculate the total profit of conventional generators and farms (in day‐ahead and balancing), providing that the balancing settlement is 1) one‐price, and 2) two‐price.

In [1]:
# Import Data
## Conventional Generators data
Conventional_generators = [
        1 2 7 13 15 15 16 18 21 22 23 23;
        13.32 13.32 20.7 20.93 26.11 10.52 10.52 6.02 5.47 7 10.52 10.89;
        1.68 1.68 3.30 4.07 1.89 5.48 5.48 4.98 5.53 8.00 3.45 5.11;
        2.32 2.32 4.67 3.93 3.11 3.52 3.52 5.02 4.97 6.00 2.52 2.89;
        106.4 106.4 245 413.7 42 108.5 108.5 280 280 210 217 245;
        48 48 84 216 42 36 36 60 60 48 72 48;
        48 48 84 216 42 36 36 60 60 48 72 48;
    ]

##  Wind Farms data
Wind_farms = [
    3 5 16 21;
    500 500 300 300;
    120.54 115.52 53.34 38.16;
]

## Demands data
Demands = [
    1   2   3   4   5   6 7 8 9 10 13 14 15 16 18 19 20;
    84  75  139 58  55  106 97  132 135 150 205 150 245 77  258 141 100;
    500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500;
    ]

## Transmission_lines Data
Transmission_fir = [
    1 1 1 2 2 3 3 4 5 6 7 8 8 9 9 10 10;
    2 3 5 4 6 9 24 9 10 10 8 9 10 11 12 11 12;
    0.0146 0.2253 0.0907 0.1356 0.205 0.1271 0.084 0.111 0.094 0.0642 0.0652 0.1762 0.1762 0.084 0.084 0.084 0.084;
    175 175 350 175 175 175 400 175 350 175 350 175 175 400 400 400 400;
]
Transmission_sec =[
    11 11 12 12 13 14 15 15 15 16 16 17 17 18 19 20 21;
    13 14 13 23 23 16 16 21 24 17 19 18 22 21 20 23 22;
    0.0488 0.0426 0.0488 0.0985 0.0884 0.0594 0.0172 0.0249 0.0529 0.0263 0.0234 0.0143 0.1069 0.0132 0.0203 0.0112 0.0692;
    500 500 500 500 250 250 500 400 500 500 500 500 500 1000 1000 1000 500;
]
Transmission_lines = hcat(Transmission_fir, Transmission_sec)

# Get number of elements connected to the grid from Data

N_G = Int64.(size(Conventional_generators,2))       # number of Conventional_generators
N_W = Int64.(size(Wind_farms,2))                    # number of Wind_farms
N_D = Int64.(size(Demands,2))                               # number of Demands
N_l = Int64.(size(Transmission_lines,2) )                   # number of Transmission_lines
N_n = Int64.(findmax(Transmission_lines[1:2,:])[1]) # number of nodes
N_a = 2                                             # number of zones

## Get information from Input 

# conventional generators
cg_location = Int64.(Conventional_generators[1,:])          # Node location of conventional generators [Node]
cg_production_cost = Conventional_generators[2,:]       # Production cost of conventional generators [$/MWh]
cg_upward_reserve_cost = Conventional_generators[3,:]   # Upward reserve cost of conventional generators [$/MWh]
cg_downward_reserve_cost = Conventional_generators[4,:] # Downward reserve cost of conventional generators [$/MWh]
cg_capacity = Conventional_generators[5,:]              # [MW]
cg_max_upward_reserve_provision_capability = Conventional_generators[6,:]   # [MW]
cg_max_downward_reserve_provision_capability = Conventional_generators[7,:] # [MW]

# wind farm
wf_location = Int64.(Wind_farms[1,:])       # [Node]
wf_installed_capacity = Wind_farms[2,:] # [MW]
wf_day_ahead_forecast = Wind_farms[3,:] # [MWh]

# demands
d_location = Int64.(Demands[1,:])    # [Node]
d_consumption = Demands[2,:]         # [MWh]
d_curtaliment_cost = Demands[3,:]    # [$/MWh]

# transmission lines
##
tl_from_node = Transmission_lines[1,:]
tl_to_node = Transmission_lines[2,:]
tl_impedance = Transmission_lines[3,:]
tl_capacity = Transmission_lines[4,:]
tl_susceptance = 1 ./ Transmission_lines[3,:];

## 1. Clear the market price with inelastic demand

\begin{aligned}
\min_{p^{G}_g,p^{W}_w} \quad & \sum_{g=1}^{N_{G}}{C_{g}p^{G}_g} + \sum_{w=1}^{N_{W}}{T_{w} p^{W}_w}\\
\textrm{s.t.} \quad & 0 \geq p^{G}_{g} \geq \bar{P^{G}_{g}} \quad \quad \forall g\\
  & 0 \geq p^{W}_{w} \geq \bar{P^{W}_{w}} \quad \quad \forall w \\
  & \sum_{d=1}^{N_{D}}{p^{D}_d} - \sum_{g=1}^{N_{G}}{p^{G}_g} - \sum_{w=1}^{N_{W}}{p^{W}_w} = 0 \quad :\lambda \\
\end{aligned}

In [133]:
using JuMP
using GLPK

# variable constraints parameters
P̅ᴳ = cg_capacity
P̲ᴳ = zeros(N_G)
P̅ᵂ = wf_day_ahead_forecast
P̲ᵂ = zeros(N_W)

# cost parameters
C = (cg_capacity/2)'
T = zeros(4)'

# inelastic demands parameters
pᴰ= Demands[2,:] # Demends inelastic price

inelastic_model = Model(GLPK.Optimizer)

@variable(inelastic_model, pᴳ[1:N_G])
@variable(inelastic_model, pᵂ[1:N_W])

@constraint(inelastic_model, cg_con[i=1:N_G], P̲ᴳ[i] <= pᴳ[i] <= P̅ᴳ[i])
@constraint(inelastic_model, wt_con[i=1:N_W], P̲ᵂ[i] <= pᵂ[i] <= P̅ᵂ[i])

@constraint(inelastic_model, balance_con, + sum(pᴰ) - sum(pᴳ) - sum(pᵂ) .== 0)

@objective(inelastic_model, Min, T*pᵂ + C*pᴳ)

#print(inelastic_model)
optimize!(inelastic_model)
@show termination_status(inelastic_model)
@show primal_status(inelastic_model)
@show dual_status(inelastic_model)
@show objective_value(inelastic_model)
@show value.(pᴳ)
@show value.(pᵂ)

MCP_𝒟𝒜 = abs(shadow_price(balance_con)) # Market Clearing Price

pᴳ_𝒟𝒜 = value.(pᴳ) # p\scrD\scrA\^G
pᵂ_𝒟𝒜 = value.(pᵂ)
pᴰ_𝒟𝒜 = value.(pᴰ)

termination_status(vector_model) = MathOptInterface.OPTIMAL
primal_status(vector_model) = MathOptInterface.FEASIBLE_POINT
dual_status(vector_model) = MathOptInterface.FEASIBLE_POINT
objective_value(vector_model) = 198284.31
value.(pᴳ) = [106.4, 106.4, 245.0, 0.0, 42.0, 108.5, 108.5, 210.6400000000001, 280.0, 210.0, 217.0, 245.0]
value.(pᵂ) = [120.54, 115.52, 53.34, 38.16]


4-element Vector{Float64}:
 16875.600000000002
 16172.8
  7467.6
  5342.4

## 2. Balancing stage

In the balancing stage, generator 7 becomes out of the system (unexpected outage). The actual
production of farms 1 and 2 is 10% lower than their day‐ahead forecast, while that of farms 3 and 4 is
25% higher than their day‐ahead forecast. All remaining 11 conventional generators (but not
demands) are potential balancing service providers. There is no other balancing service provider
(except for involuntarily load curtailment). Each conventional generator as the balancing service
provider offers the upward balancing service at a price equal to the day‐ahead price plus 10% of her
production cost. Similarly, she offers the downward balancing service (if she can depending on her
day‐ahead schedule) at a price equal to the day‐ahead price minus 12% of her production cost. Clear
the balancing market, and derive the balancing price.

In [134]:
## System Imbalance calculation
pᴳ_𝒜 = copy(value.(pᴳ))
pᵂ_𝒜 = copy(value.(pᵂ))
pᴰ_𝒜 = copy(value.(pᴰ))

# generator 7 becomes out of the system (unexpected outage)
pᴳ_𝒜[7] = 0

# The actual production of wind farms 1 and 2 is 10% lower than their day‐ahead forecast
pᵂ_𝒜[1:2] = copy(P̅ᵂ[1:2]) * 0.9 

# The actual production of wind farms 3 and 4 is 25% higher than their day‐ahead forecast
pᵂ_𝒜[3:4] = copy(P̅ᵂ[3:4]) * 1.25; 

# need balance
ΔP = - sum(pᴳ_𝒜 - pᴳ_𝒟𝒜) - sum(pᵂ_𝒜 - pᵂ_𝒟𝒜) + sum(pᴰ_𝒜 - pᴰ_𝒟𝒜)

## Power Reserve Costraints
# All remaining 11 conventional generators (but not demands) are potential balancing service providers.
# There is no other balancing service provider (except for involuntarily load curtailment).
αᴳ = Bool.((ones(N_G)))
x = findall(pᴳ_𝒜 -> pᴳ_𝒜 == 0, pᴳ_𝒜)
αᴳ[x] .= 0

P̅ᴳ_up = αᴳ .* (cg_capacity - pᴳ_𝒜)
P̅ᴳ_down = copy(pᴳ_𝒜)
P̲ᴳ_up = zeros(N_G)
P̲ᴳ_down = zeros(N_G)

P̅ᵂ_up = zeros(N_W)
P̅ᵂ_down = zeros(N_W)
P̲ᵂ_up = zeros(N_W)
P̲ᵂ_down = zeros(N_W)

P̅ᴰ_up = copy(pᴰ_𝒜)
P̅ᴰ_down = zeros(N_D)
P̲ᴰ_up = zeros(N_D)
P̲ᴰ_down = zeros(N_D)

## Costs
# Conventional generator as the balancing service provider offers the upward balancing
# service at a price equal to the day‐ahead price plus 10% of her production cost
Cᴳ_up = C + cg_production_cost' * 0.1  
# she offers the downward balancing service (if she can depending on her day‐ahead schedule)
# at a price equal to the day‐ahead price minus 12% of her production cost
Cᴳ_down = C - cg_production_cost' * 0.12 

# load curtailment cost
Cᴰ_up = d_curtaliment_cost'
# load flexible
Cᴰ_down = ones(N_D)'

Cᵂ_up = ones(N_W)'
Cᵂ_down = ones(N_W)';

In [138]:
bal_mrkt_model = Model(GLPK.Optimizer)

@variable(bal_mrkt_model, pᴳ_up[1:N_G])
@variable(bal_mrkt_model, pᴳ_down[1:N_G])

@variable(bal_mrkt_model, pᵂ_up[1:N_W])
@variable(bal_mrkt_model, pᵂ_down[1:N_W])

@variable(bal_mrkt_model, pᴰ_up[1:N_D])
@variable(bal_mrkt_model, pᴰ_down[1:N_D])

@constraint(bal_mrkt_model,   cg_p_up_con[i=1:N_G], P̲ᴳ_up[i]   <= pᴳ_up[i]   <= P̅ᴳ_up[i])
@constraint(bal_mrkt_model, cg_p_down_con[i=1:N_G], P̲ᴳ_down[i] <= pᴳ_down[i] <= P̅ᴳ_down[i])

@constraint(bal_mrkt_model,   wt_p_up_con[i=1:N_W], P̲ᵂ_up[i]   <= pᵂ_up[i]   <= P̅ᵂ_up[i])
@constraint(bal_mrkt_model, wt_p_down_con[i=1:N_W], P̲ᵂ_down[i] <= pᵂ_down[i] <= P̅ᵂ_down[i])

@constraint(bal_mrkt_model, d_p_up_con[i=1:N_D], P̲ᴰ_up[i] <= pᴰ_up[i] <= P̅ᴰ_up[i])
@constraint(bal_mrkt_model, d_p_down_con[i=1:N_D], P̲ᴰ_down[i] <= pᴰ_down[i] <= P̅ᴰ_down[i])

@constraint(bal_mrkt_model, balance_con, sum(pᴰ_up - pᴰ_down) + sum(pᴳ_up - pᴳ_down) + sum(pᵂ_up - pᵂ_down) .== ΔP)

@objective(bal_mrkt_model, Min, + Cᴳ_up * pᴳ_up - Cᴳ_down * pᴳ_down + Cᵂ_up * pᵂ_up - Cᵂ_down * pᵂ_down + Cᴰ_up * pᴰ_up - Cᴰ_down * pᴰ_down)

#print(bal_mrkt_model)
optimize!(bal_mrkt_model)
@show solve_time(bal_mrkt_model)
@show termination_status(bal_mrkt_model)
@show primal_status(bal_mrkt_model)
@show dual_status(bal_mrkt_model)
@show objective_value(bal_mrkt_model)
@show value.(pᴳ_up)
@show value.(pᴳ_down)
@show value.(pᴰ_up)
@show value.(pᴰ_down)

MCP_ℬ = abs(shadow_price(balance_con)) # Market Clearing Price

solve_time(bal_mrkt_model) = 0.0001270771026611328
termination_status(bal_mrkt_model) = MathOptInterface.OPTIMAL
primal_status(bal_mrkt_model) = MathOptInterface.FEASIBLE_POINT
dual_status(bal_mrkt_model) = MathOptInterface.FEASIBLE_POINT
objective_value(bal_mrkt_model) = 29687.654720000035
value.(pᴳ_up) = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 69.3599999999999, 0.0, 0.0, 0.0, 0.0]
value.(pᴳ_down) = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
value.(pᴰ_up) = [39.871000000000095, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
value.(pᴰ_down) = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


500.0

## 3. Calculate the total profit 
Calculate the total profit of conventional generators and farms (in day‐ahead and balancing), providing that the balancing settlement is 1) one‐price, and 2) two‐price.

In [175]:
# Total Profit in Day-ahead
Rᴳ_𝒟𝒜 = value.(pᴳ) * MCP_𝒟𝒜 # Revenue of conventional generators
cost_fuel_𝒟𝒜 = cg_production_cost .* value.(pᴳ)
Profitᴳ_𝒟𝒜 = Rᴳ_𝒟𝒜 - cost_fuel_𝒟𝒜

Rᵂ_𝒟𝒜 = value.(pᵂ) * MCP_𝒟𝒜 # Revenue of wind farms
Profitᵂ_𝒟𝒜 = Rᵂ_𝒟𝒜 #Profit

# Total Profit in Balancing

## From balancing service providers side:
## BSP - Balancing Services Provider
Rᴳ_ℬ = value.(pᴳ_up) * MCP_ℬ
cost_fuel_ℬ = cg_production_cost .* value.(pᴳ_up)
BSP_Profitᴳ_ℬ = Rᴳ_ℬ - cost_fuel_ℬ

## From the side of agents who caused the imbalance:
### if  balancing settlement is One-Price
### IA_op - Imbalance Agent one price
IA_op_Profitᴳ_ℬ = (pᴳ_𝒜 - pᴳ_𝒟𝒜) * MCP_ℬ 
IA_op_Profitᵂ_ℬ = (pᵂ_𝒜 - pᵂ_𝒟𝒜) * MCP_ℬ 

### if  balancing settlement is One-Price
### IA_op - Imbalance Agent one price
IA_tp_Profitᴳ_ℬ = zeros(N_G)
for i in 1:N_G
    if pᴳ_𝒜[i] - pᴳ_𝒟𝒜[i]>= 0
        IA_tp_Profitᴳ_ℬ[i] = (pᴳ_𝒜[i] - pᴳ_𝒟𝒜[i]) * MCP_𝒟𝒜
    else 
        IA_tp_Profitᴳ_ℬ[i] = (pᴳ_𝒜[i] - pᴳ_𝒟𝒜[i]) * MCP_ℬ 
    end
end

IA_tp_Profitᵂ_ℬ =zeros(N_W)
for i in 1:N_W
    if (pᵂ_𝒜[i] - pᵂ_𝒟𝒜[i]) >= 0
        IA_tp_Profitᵂ_ℬ[i] = (pᵂ_𝒜[i] - pᵂ_𝒟𝒜[i]) * MCP_𝒟𝒜
    else 
        IA_tp_Profitᵂ_ℬ[i] = (pᵂ_𝒜[i] - pᵂ_𝒟𝒜[i]) * MCP_ℬ 
    end
end

## Total Profit if balancing settlement is One-Price
op_Profitᴳ_ℬ = BSP_Profitᴳ_ℬ + IA_op_Profitᴳ_ℬ
op_Profitᵂ_ℬ = IA_op_Profitᵂ_ℬ

## Total Profit if balancing settlement is One-Price
tp_Profitᴳ_ℬ = BSP_Profitᴳ_ℬ + IA_tp_Profitᴳ_ℬ
tp_Profitᵂ_ℬ = IA_tp_Profitᵂ_ℬ;